In [1]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
from pathlib import Path

/home/chewye/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
path = '../../prepare_data/data/base_line.csv'
df = pd.read_csv(Path(path).resolve())
df.drop('match_id', axis=1, inplace=True)
df.head()

,gameweek_gameweek,gameweek_compSeason_label,teams_team_1_name,teams_team_2_name,team_1_hue
0,38,2023,Arsenal,Everton,1
1,38,2023,Brentford,Newcastle United,0
2,38,2023,Brighton & Hove Albion,Manchester United,0
3,38,2023,Burnley,Nottingham Forest,0
4,38,2023,Chelsea,Bournemouth,1


In [3]:
df[df['gameweek_compSeason_label']==2023]

,gameweek_gameweek,gameweek_compSeason_label,teams_team_1_name,teams_team_2_name,team_1_hue
0,38,2023,Arsenal,Everton,1
1,38,2023,Brentford,Newcastle United,0
2,38,2023,Brighton & Hove Albion,Manchester United,0
3,38,2023,Burnley,Nottingham Forest,0
4,38,2023,Chelsea,Bournemouth,1
...,...,...,...,...,...
375,1,2023,Brighton & Hove Albion,Luton Town,1
376,1,2023,Everton,Fulham,0
377,1,2023,Sheffield United,Crystal Palace,0
378,1,2023,Arsenal,Nottingham Forest,1


In [4]:
train = df[df['gameweek_compSeason_label']==2023]
val = df[df['gameweek_compSeason_label']!=2023]

In [5]:
train

,gameweek_gameweek,gameweek_compSeason_label,teams_team_1_name,teams_team_2_name,team_1_hue
0,38,2023,Arsenal,Everton,1
1,38,2023,Brentford,Newcastle United,0
2,38,2023,Brighton & Hove Albion,Manchester United,0
3,38,2023,Burnley,Nottingham Forest,0
4,38,2023,Chelsea,Bournemouth,1
...,...,...,...,...,...
375,1,2023,Brighton & Hove Albion,Luton Town,1
376,1,2023,Everton,Fulham,0
377,1,2023,Sheffield United,Crystal Palace,0
378,1,2023,Arsenal,Nottingham Forest,1


In [6]:
y_train = train['team_1_hue']
x_train = train.drop('team_1_hue', axis=1)

y_val = val['team_1_hue']
x_val = val.drop('team_1_hue', axis=1)

In [7]:
model_baseline = CatBoostClassifier()

In [8]:
model_baseline.fit(x_train, y_train, cat_features=list(x_train.columns), verbose=False)

In [9]:
pred = model_baseline.predict(x_val)
pred_proba = model_baseline.predict_proba(x_val)
y_val = y_val.to_numpy().reshape(-1, 1)

In [10]:
pred = np.concatenate((y_val, pred, pred_proba), axis=1)
df_pred = pd.DataFrame(pred, columns=['true', 'pred', '0_prob', '1_prob', '2_prob'])

In [11]:
df_pred['error'] = np.where(df_pred['true']==df_pred['pred'], 0, 1)
df_pred.head()


,true,pred,0_prob,1_prob,2_prob,error
0,1.0,1.0,0.102275,0.771593,0.126133,0
1,1.0,1.0,0.064093,0.923300,0.012607,0
2,1.0,0.0,0.845546,0.136974,0.017479,1
3,2.0,1.0,0.319846,0.513404,0.166750,1
4,2.0,1.0,0.247348,0.708906,0.043746,1


In [12]:
df_pred['error'].sum()

1964

In [13]:
print(f"Accuracy: {accuracy_score(df_pred['true'], df_pred['pred'])}")
print(f"F1 weighted: {f1_score(df_pred['true'], df_pred['pred'], average='weighted')}")
print(f"F1 macro: {f1_score(df_pred['true'], df_pred['pred'], average='macro')}")

Accuracy: 0.4257309941520468
F1 weighted: 0.3968672053929261
F1 macro: 0.3524405231985399
